In [32]:
import pandas as pd
from openai import OpenAI
from dataclasses import dataclass
from typing import List
import numpy as np
from collections import Counter
import re

In [48]:
@dataclass
class InformationStructure:
    information: str
    embedding: List[float]


information = [
    'Proběhla první onkologická konzultace.', 'Pacientka byla diagnostikována s karcinomem levého prsu.', 'Pacientka je po menopauze.', 'Nález se nachází v horním zevním kvadrantu levého prsu.', 'Na nálezu je kožní infiltrace.', 'V axile jsou hmatné uzliny.', 'Pacientka má dlouhodobé bolesti zad.', 'Pacientka má bolesti kloubů.', 'Pacientka má známou revmatoidní artritidu.', 'Pacientka používá francouzské hole při chůzi.', 'Bylo provedeno základní fyzikální vyšetření.', 'Proběhla kontrola dokumentace z mamologického pracoviště.', 'Závěr vyšetření uvádí suspektně lokálně pokročilý nádor prsu.', 'Je nutné dovyšetření.', 'Je nutný kompletní staging.',
    'Byla provedena klinicko-farmaceutická konzultace.', 'Byla zhodnocena chronická medikace včetně imunosupresiv.', 'Byla zhodnocena chronická medikace včetně analgetik.', 'Byla zhodnocena chronická medikace včetně suplementů.', 'Byla zjištěna snížená funkce jediné ledviny.', 'Byla zjištěna zvýšená hodnota kalcia.', 'Nebyly nalezeny zásadní interakce s plánovanou protinádorovou léčbou.', 'Bylo doporučeno sledování renálních funkcí.', 'Bylo doporučeno sledování elektrolytů.', 'Byla doporučena úprava dávek analgetik podle aktuálního stavu.',
    'Pacientka byla přijata na krátkodobý příjem na onkologickém oddělení kvůli asymptomatické hyperkalcémii.', 'Pacientka má metastatický karcinom prsu.', 'Pacientka má suspektní kostní postižení.', 'Byla provedena hydratace intravenózními infuzemi.', 'Podán byl redukovaný bolus bisfosfonátu vzhledem k monoledvině.', 'Kalcémie po terapii klesla.', 'Pacientka je klinicky stabilní.', 'Pacientka neměla dušnost.', 'Pacientka neměla febrilie.', 'Pacientka byla propuštěna domů.', 'Doporučen byl dostatečný příjem tekutin.', 'Doporučeno vysadit volně prodejné kalcium.',
    'Pacientka má HER2-pozitivní karcinom levého prsu.', 'Má diseminaci do skeletu.', 'Má diseminaci do jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Pacientka nemá nauzeu ani zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacientka kardiopulmonálně kompenzovaná.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axila obsahuje drobné tuhé uzliny.', 'Laboratorně je krevní obraz přiměřený.', 'Laboratorně jsou jaterní testy mírně zvýšené.', 'Pokračovat v aktuální léčbě.', 'Je nutné pečlivě monitorovat toxicitu léčby.', 'Je nutné pečlivě monitorovat bolest.',
    'Má HER2‑pozitivní karcinom levého prsu.', 'Má metastázy do skeletu.', 'Má metastázy do jater.', 'Pacientka je léčena paliativní systémovou léčbou.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Subjektivně uvádí kolísající bolesti kostí.', 'Pacient nemá významnou nauzeu.', 'Pacient nemá zvracení.', 'Pacient nemá klidovou dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaný.', 'V prsu je lokální rozsah tumoru stacionární.', 'V axille jsou drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě.', 'Závěr: bude probíhat pečlivé monitorování toxicity léčby.', 'Závěr: bude probíhat pečlivé monitorování bolesti.',
    'Pacient je léčen cílenou anti-HER2 terapií.', 'Transtorakální echokardiografie byla provedena v rámci monitorace kardiální funkce.', 'Levá komora má normální velikost.', 'Globální systolická funkce levé komory je zachována.', 'Ejekční frakce byla odhadnuta na přibližně 60 %.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Nebyly známky plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Kardiologicky není kontraindikace pokračovat v současném režimu léčby.', 'Doporučeno je pravidelné echokardiografické sledování.',
    'Případ byl projednán multidisciplinárním týmem bez přítomnosti pacientky.', 'Pacientka nebyla přítomna při jednání.', 'Má HER2 pozitivní karcinom levého prsu.', 'Postižení prsu je bifokální s kožní infiltrací.', 'Má mnohočetné kostní metastázy.', 'Má jaterní metastázy.', 'Není primárně indikována chirurgická léčba prsu.', 'Skelet je symptomaticky postižen.', 'Komise se shodla na paliativní systémové terapii.', 'Systémová terapie bude kombinací taxanu a dvojité blokády HER2.', 'K terapii bude doplněna kostně cílená léčba.',
    'Má HER2-pozitivní karcinom levého prsu s metastázemi do skeletu a do jater.', 'Pacientka podstupuje paliativní systémovou léčbu.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v oblasti páteře a pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Nemá významnou nauzeu ani zvracení.', 'Není přítomna klidová dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaná.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axilární uzliny jsou drobné a tuhé.', 'Laboratorně je krevní obraz přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě s pečlivým monitorováním toxicity a bolesti.',
    'Zhodnocena chronická medikace u pacienta s revmatoidní artritidou, včetně imunosupresiv.', 'Zhodnocena analgetika a suplementy.', 'Bylo upozorněno na sníženou funkci jediné ledviny.', 'Byla zjištěna zvýšená hladina kalcia.', 'Nebyly nalezeny zásadní interakce s plánovanou protinádorovou léčbou.', 'Bylo doporučeno sledovat renální funkce.', 'Bylo doporučeno sledovat elektrolyty.', 'Byla upravena dávka analgetik podle aktuálního stavu.',
    'Proběhlo komplexní onkologické vyšetření při kratší hospitalizaci.', 'Proběhla rekapitulace osobní anamnézy.', 'Proběhla rekapitulace rodinné anamnézy.', 'Proběhla rekapitulace gynekologické anamnézy.', 'Pacient měl dříve odstraněnou jednu ledvinu pro renální tumor.', 'Pacient má dlouhodobé revmatologické onemocnění.', 'Pacient má kostní metastázy karcinomu prsu.', 'Pacient má jaterní metastázy karcinomu prsu.', 'Pacient má osteoporózu.', 'Pacient má chronickou bolest zad.', 'Pokračování paliativní léčby bylo domluveno.', 'Byly domluveny pravidelné kontroly v onkologické ambulanci.', 'Proběhla krátká hospitalizace.',
    'Onkologická kontrola v průběhu paliativní systémové léčby pro HER2 pozitivní karcinom levého prsu s diseminací do skeletu a jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři a v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka udává kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacientka celkově kardiopulmonálně kompenzovaná.', 'Lokálně v prsu je stacionární rozsah tumoru.', 'Axila obsahuje drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Pacientka má pokračovat v aktuální léčbě za pečlivého monitorování toxicity a bolesti.',
    'Chirurgická ambulance provedla kontrolu po implantaci centrálního žilního portu.', 'Port byl implantován do oblasti horní hrudní stěny.', 'Lokalita portu je klidná.', 'Lokalita portu je bez zarudnutí.', 'Lokalita portu je bez sekrece.', 'Port je dobře palpačně přístupný.', 'Port je bez známek infekce.', 'Stehy byly odstraněny.', 'Pacientka byla poučena o péči o port.', 'Pacientka byla poučena o hlášení případných komplikací.', 'Hlášené případné komplikace zahrnují bolest.', 'Hlášené případné komplikace zahrnují zarudnutí.', 'Hlášené případné komplikace zahrnují febrilie.', 'Port byl ponechán k dlouhodobému používání pro systémovou léčbu.',
    'Diagnóza je HER2‑pozitivní karcinom levého prsu.', 'Diseminace do skeletu.', 'Diseminace do jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka uvádí kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacient kardiopulmonálně kompenzovaný.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axila obsahuje drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Pokračování v aktuální léčbě.', 'Bude prováděno pečlivé monitorování toxicity léčby.', 'Bude prováděno pečlivé monitorování bolesti.',
    'Pacientka má HER2‑pozitivní karcinom levého prsu.', 'Karcinom levého prsu má metastázy do skeletu.', 'Karcinom levého prsu se rozšířil do jater.', 'Pacientka je v paliativní systémové léčbě.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Pacientka je kardiopulmonálně kompenzovaná.', 'V prsu je stacionární rozsah tumoru.', 'V axile jsou drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě.', 'Závěr: monitorovat toxické účinky léčby.', 'Závěr: monitorovat bolest.',
    'Transtorakální echokardiografie byla provedena v rámci monitorace kardiální funkce při podané cílené anti-HER2 léčbě.', 'Levá komora má normální velikost.', 'Globální systolická funkce je zachována.', 'Ejekční frakce odhadovaná kolem 60 %.', 'Nevýznamná regurgitace na aortální chlopni.', 'Nevýznamná regurgitace na mitrální chlopni.', 'Není zjištěna plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Závěr: kardiologicky bez kontraindikace pokračování v současném režimu léčby.', 'Doporučeno pravidelné echokardiografické sledování.',
    'Má HER2 pozitivní karcinom levého prsu.', 'Karcinom levého prsu metastazuje do skeletu.', 'Karcinom levého prsu metastazuje do jater.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně udává kolísající bolesti kostí.', 'Pacientka nemá nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Kardiopulmonální status je kompenzovaný.', 'Lokální rozsah tumoru v prsu je stacionární.', 'Axilární uzliny jsou drobně tuhé.', 'Krevní obraz je přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Pokračuje se v aktuální léčbě.', 'Je nutné pečlivé monitorování toxicity léčby.', 'Je nutné pečlivé monitorování bolesti.',
    'Spondylo-onkologická komise provedla konzultaci nálezu na páteři a pánevním skeletu.', 'Na zobrazovacím vyšetření byly popsány mnohočetná osteolytická ložiska.', 'Na zobrazovacím vyšetření byly popsány kompresivní fraktury obratlových těl.', 'Bylo identifikováno riziko patologické fraktury proximálního femuru.', 'Bylo doporučeno šetrné zatěžování.', 'Bylo doporučeno používání ortopedických pomůcek.', 'Bylo doporučeno zvážení stabilizačního výkonu.', 'Bylo doporučeno radioterapie na symptomatické úseky páteře.', 'Bylo doporučeno pokračování kostně cílené léčby.',
    'Proběhla telefonická konzultace upřesňující termíny kontrol a laboratorních odběrů.', 'Proběhlo probrání průběhu posledních dní se pacientkou nebo s rodinným příslušníkem.', 'Pacientka neměla horečku.', 'Nebyly zaznamenány nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno je nadále sledovat teplotu.', 'Doporučeno je kontaktovat pracoviště při náhlém zhoršení bolestí nebo při pádu.', 'Doporučeno je dostavit se k plánované kontrole včetně krevního obrazu a biochemie.',
    'Pacientka má HER2 pozitivní karcinom levého prsu.', 'Karcinom má diseminaci do skeletu.', 'Karcinom má diseminaci do jater.', 'Pacientka je na paliativní systémové léčbě.', 'Probíhá onkologická kontrola.', 'Pacientka přichází na vozíku.', 'Bolesti v páteři omezují mobilitu.', 'Bolesti v oblasti pánevního skeletu omezují mobilitu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně uvádí kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaná.', 'Rozsah tumoru v prsu je stacionární.', 'Axilární uzliny jsou drobně tuhé.', 'Laboratorní krevní obraz je přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr je pokračovat v aktuální léčbě.', 'Bude prováděno pečlivé monitorování toxicity léčby.', 'Bolest bude pečlivě monitorována.',
    'Diagnóza je HER2 pozitivní karcinom levého prsu s diseminací do skeletu a jater.', 'Probíhá onkologická kontrola v průběhu paliativní systémové léčby.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři a v oblasti pánevního skeletu.', 'Zvládá základní sebeobsluhu.', 'Pacientka uvádí kolísavé bolesti kostí.', 'Pacientka nemá významnou nauzeu ani zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je kardiopulmonálně kompenzovaná.', 'V prsu je lokálně stacionární rozsah tumoru.', 'Axila obsahuje drobné tuhé uzliny.', 'Laboratorně je krevní obraz přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Závěr: pokračovat v aktuální léčbě za pečlivého monitorování toxicity a bolesti.',
    'Transtorakální echokardiografie byla provedena v rámci monitorace kardiální funkce při podané cílené anti-HER2 léčbě.', 'Levá komora má normální velikost.', 'Levá komora má zachovanou globální systolickou funkci.', 'Ejekční frakce byla odhadována kolem 60 procent.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Nebyly zjištěny známky plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Závěr uvádí, že není kardiologicky kontraindikováno pokračovat v současném režimu léčby.', 'Doporučeno je pravidelné echokardiografické sledování.',
    'Pacientka má HER2 pozitivní karcinom levého prsu.', 'Karcinom levého prsu má diseminaci do skeletu.', 'Karcinom levého prsu má diseminaci do jater.', 'Pacientka je na onkologické kontrole.', 'Pacientka přijíždí na vozíku.', 'Mobilita je omezena bolestmi v páteři.', 'Mobilita je omezena bolestmi v oblasti pánevního skeletu.', 'Pacientka zvládá základní sebeobsluhu.', 'Pacientka subjektivně udává kolísající bolesti kostí.', 'Pacientka nemá významnou nauzeu.', 'Pacientka nemá zvracení.', 'Pacientka nemá klidovou dušnost.', 'Fyzikálně je pacientka kardiopulmonálně kompenzovaná.', 'Lokálně v prsu je stacionární rozsah tumoru.', 'Axila obsahuje drobné tuhé uzliny.', 'Krevní obraz je laboratorně přiměřený.', 'Jaterní testy jsou mírně zvýšené.', 'Bude pokračovat v aktuální léčbě.', 'Bude prováděno pečlivé monitorování toxicity a bolesti.',
    'Byla provedena transtorakální echokardiografie pro monitoraci kardiální funkce během cílené anti-HER2 léčby.', 'Levá komora má normální velikost.', 'Globální systolická funkce je zachována.', 'Ejekční frakce odhadovaná kolem 60 %.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Nebyly zjištěny známky plicní hypertenze.', 'Perikard je bez patologického výpotku.', 'Závěr zní, že kardiologicky není kontraindikace pokračování v současném režimu léčby.', 'Doporučeno pravidelné echokardiografické sledování.',
    'Pacientka je léčená pro metastatický karcinom prsu.', 'Posouzen byl celkový interní stav.', 'Krevní tlak byl mírně zvýšený.', 'Pacientka je oběhově kompenzovaná.', 'Laboratorně jsou renální parametry mírně zhoršené, odpovídající chronickému stavu po nefrektomii.', 'Nebyly zjištěny překážky pro pokračování v protinádorové léčbě.', 'Byla doporučena úprava antihypertenzní terapie.', 'Byly doporučeny pravidelné kontroly u praktického lékaře.',
    'Pacientka má HER2 pozitivní karcinom levého prsu s diseminací do skeletu a do jater.', ' Pacientka je na paliativní systémové léčbě.', ' Pacientka je na onkologické kontrole.', ' Pacientka přijíždí na vozíku.', ' Mobilita je omezena bolestmi v páteři a v oblasti pánevního skeletu.', ' Zvládá základní sebeobsluhu.', ' Subjektivně udává kolísající bolesti kostí.', ' Nemá významnou nauzeu ani zvracení.', ' Není přítomna klidová dušnost.', ' Fyzicky je kardiopulmonálně kompenzovaná.', ' Lokálně v prsu je stacionární rozsah tumoru.', ' Axila obsahuje drobné tuhé uzliny.', ' Laboratorně je krevní obraz přiměřený.', ' Mírně zvýšené jaterní testy.', ' Závěr zní, že je třeba pokračovat v aktuální léčbě.', ' Léčba bude prováděna s pečlivým monitorováním toxicity a bolesti.',
    'Proběhla onko-hematologická konzultace po zobrazení mozku a dalších částí skeletu.', 'Byla potvrzena diseminace onemocnění s postižením mozkového parenchymu a mnohačetnými kostními metastázami.', 'Onko-hematolog doporučuje zachovat stávající systémovou léčbu.', 'Onko-hematolog doporučuje soustředit se na symptomatickou kontrolu bolesti a neurologických obtíží.', 'Případně doplnit lokální radioterapii na nejvíce symptomatická ložiska.',
    'Byla provedena klinicko-farmaceutická konzultace.', 'Zhodnocena byla chronická medikace včetně imunosupresiv při revmatoidní artritidě.', 'Zhodnocena byla chronická medikace obsahující analgetika.', 'Zhodnocena byla chronická medikace obsahující suplementy.', 'Bylo upozorněno na sníženou funkci jediné ledviny.', 'Bylo upozorněno na zvýšenou hodnotu kalcia.', 'Nebyla nalezena žádná zásadní interakce s plánovanou protinádorovou léčbou.', 'Bylo doporučeno sledování renálních funkcí.', 'Bylo doporučeno sledování elektrolytů.', 'Bylo doporučeno upravit dávky analgetik podle aktuálního stavu.',
    'Proběhla konzultace paliativního týmu.', 'Byla přítomna pacientka.', 'Byla přítomna blízká osoba.', 'Diskutovány byly možnosti domácí péče.', 'Probíhá úprava prostředí s ohledem na prevenci pádů.', 'Probíhá plánování budoucí léčby.', 'Bolesti kostí jsou hlavní obtíží.', 'Pacientka má únavu.', 'Pacientka má obavy z dalšího průběhu onemocnění.', 'Byla navržena optimalizace analgetické terapie.', 'Bylo zváženo zapojení domácí ošetřovatelské služby.', 'Byla poskytována podpora při zvládání psychické zátěže.',
    'Ambulantní kontrola probíhá po nedávné hospitalizaci a po úpravě analgetické léčby.', 'Pacientka popisuje mírné zlepšení zvládání běžných denních aktivit.', 'Přetrvává únava.', 'Přetrvává omezená chůze.', 'Nejsou žádné nové krvácivé projevy.', 'Stolice je bez patologie.', 'Močení je bez patologie.', 'Bylo dohodnuto pokračování v aktuálním analgetickém režimu.', 'Kontrola proběhne za několik týdnů.', 'V případě náhlého zhoršení stavu bude provedena kontrola dříve.',
    'Proběhla konzultace paliativního týmu.', 'Byla přítomna pacientka.', 'Byla přítomna blízká osoba.', 'Diskutovány byly možnosti domácí péče.', 'Diskutována byla úprava prostředí z hlediska prevence pádů.', 'Diskutováno bylo plánování budoucí léčby.', 'Hlavními obtížemi jsou bolesti kostí.', 'Hlavními obtížemi je únava.', 'Hlavními obtížemi jsou obavy z dalšího průběhu onemocnění.', 'Byla navržena optimalizace analgetické terapie.', 'Byla navržena možnost zapojení domácí ošetřovatelské služby.', 'Byla navržena podpora při zvládání psychické zátěže.',
    'Pacientce byly telefonicky sděleny výsledky posledních vyšetření.', 'Pacientce byl sdělen další postup.', 'Onemocnění je hodnoceno jako stabilizované.', 'Pacientce hrozí vysoké riziko komplikací při pádu.', 'Pacientce hrozí vysoké riziko komplikací při infekci.', 'Je kladen důraz na kvalitní analgezii.', 'Je kladen důraz na prevenci dekubitů.', 'Je poskytována podpora rodiny v péči o pacientku.', 'Byla nabídnuta možnost další konzultace dle potřeby.',
    'Radioterapeutické vyšetření je plánováno.', 'Důvodem léčby je paliativní ozáření bolestivých kostních ložisek v oblasti páteře a pánevního skeletu.', 'Případně bude provedena radioterapie prsu.', 'Pacientka má výrazné bolesti při chůzi.', 'Pacientka má bolesti při změně polohy.', 'Cílem léčby je úleva od bolesti.', 'Cílem léčby je snížení rizika patologických fraktur.', 'Je naplánováno krátké frakcionované schéma s ohledem na celkový stav a probíhající systémovou léčbu.', 'Ozařovaná oblast zahrnuje páteř a pánevní skelet.',
    'Proběhla telefonická konzultace.', 'Byly upřesněny termíny kontrol.', 'Byly upřesněny termíny laboratorních odběrů.', 'S pacientkou byl probrán průběh posledních dní.', 'Pacientka neměla horečku.', 'Pacientka neměla nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Bylo doporučeno nadále sledovat teplotu.', 'Bylo doporučeno kontaktovat pracoviště při náhlém zhoršení bolestí.', 'Bylo doporučeno kontaktovat pracoviště při pádu.', 'Bylo doporučeno dostavit se k plánované kontrole včetně krevního obrazu a biochemie.',
    'Pacientka je po menopauze.', 'Pacientka má nově diagnostikovaný karcinom levého prsu.', 'Proběhla první onkologická konzultace.', 'Nález v horním zevním kvadrantu levého prsu s kožní infiltrací.', 'Hmatné uzliny v axile.', 'Subjektivně pacient uvádí dlouhodobé bolesti zad.', 'Subjektivně pacient uvádí bolesti kloubů při známé revmatoidní artritidě.', 'Pacientka chodí o francouzských holích.', 'Provedeno základní fyzikální vyšetření.', 'Provedena kontrola dokumentace z mamologického pracoviště.', 'Závěr vyšetření ukazuje suspektně lokálně pokročilý nádor prsu.', 'Je nutné dovyšetření.', 'Je nutné provést kompletní staging.',
    'Proběhla telefonická konzultace ohledně upřesnění termínů kontrol a laboratorních odběrů.', 'Byl probrán průběh posledních dní s pacientkou.', 'Do konverzace byl v případě potřeby zapojen rodinný příslušník.', 'Pacientka neměla horečku v posledních dnech.', 'Nebyly zaznamenány nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno nadále sledovat teplotu.', 'Doporučeno kontaktovat pracoviště při náhlém zhoršení bolestí.', 'Doporučeno kontaktovat pracoviště při pádu.', 'Doporučeno dostavit se k plánované kontrole.', 'Krevní obraz má být součástí plánované kontroly.', 'Biochemie má být součástí plánované kontroly.',
    'Transtorakální echokardiografie byla provedena pro monitoraci kardiální funkce při cílené anti-HER2 léčbě.', 'Levá komora má normální velikost.', 'Globální systolická funkce levé komory je zachována.', 'Ejekční frakce levé komory je odhadována na přibližně 60 %.', 'Regurgitace na aortální chlopni je nevýznamná.', 'Regurgitace na mitrální chlopni je nevýznamná.', 'Plicní hypertenze není přítomna.', 'Perikard je bez patologického výpotku.', 'Kardiologicky není kontraindikace pro pokračování v současném režimu léčby.', 'Bylo doporučeno pravidelné echokardiografické sledování.',
    'Proběhla telefonická konzultace s pacientkou (případně rodinným příslušníkem).', 'Byl probrán průběh posledních dní.', 'Nebyla horečka.', 'Neobjevily se nové neurologické příznaky.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno nadále sledovat teplotu.', 'Doporučeno kontaktovat pracoviště při náhlém zhoršení bolesti.', 'Doporučeno kontaktovat pracoviště při pádu.', 'Doporučeno dostavit se k plánované kontrole.', 'Doporučeno provést krevní obraz.', 'Doporučeno provést biochemii.',
    'Telefonická konzultace proběhla za účelem upřesnění termínů kontrol a laboratorních odběrů.', 'S pacientkou (případně rodinným příslušníkem) byl probrán průběh posledních dní.', 'Průběh posledních dní byl bez horeček.', 'Průběh posledních dní byl bez nových neurologických příznaků.', 'Přetrvávají bolesti zad.', 'Účinek analgetik je dostatečný.', 'Doporučeno nadále sledovat teplotu.', 'Doporučeno kontaktovat pracoviště při náhlém zhoršení bolestí nebo při pádu.', 'Doporučeno dostavit se k plánované kontrole včetně krevního obrazu a biochemie.',
    'Pacientka je nadále sledována v pravidelných intervalech.', 'Systémová léčba byla přizpůsobena aktuálnímu výkonostnímu stavu.', 'Přetrvává závislost na pomoci druhé osoby při některých denních činnostech.', 'Psychicky je pacientka stabilní.', 'Pacientka je informovaná o povaze onemocnění a o možnostech další péče.', 'Domluvena je další kontrola.', 'Je možné kontaktovat pracoviště při jakémkoli zhoršení.'
]

In [49]:
client = OpenAI()

response = client.embeddings.create(
    input=information,
    model="text-embedding-3-large"
)

In [50]:
information_nodes = []

for idx, res in enumerate(response.data):
    information_nodes.append(InformationStructure(information[idx], res.embedding))

In [51]:
def semantic_search(
    search_term: str,
    information_nodes: List[InformationStructure],
    client: OpenAI,
    threshold: float = 0.5,
    semantic_weight: float = 0.75,
    syntactic_weight: float = 0.25
) -> List[InformationStructure]:
    """
    Perform hybrid semantic search combining semantic similarity (embeddings) 
    and syntactic similarity (BM25-like).
    
    Args:
        search_term: The search query
        information_nodes: List of InformationStructure objects to search through
        client: OpenAI client for generating embeddings
        threshold: Minimum combined score to include a result (default: 0.5)
        semantic_weight: Weight for semantic similarity (default: 0.75)
        syntactic_weight: Weight for syntactic similarity (default: 0.25)
    
    Returns:
        List of InformationStructure objects that match above the threshold,
        sorted by combined score (descending)
    """
    # Get embedding for search term
    search_response = client.embeddings.create(
        input=[search_term],
        model="text-embedding-3-large"
    )
    search_embedding = np.array(search_response.data[0].embedding)
    
    # Normalize search embedding
    search_embedding_norm = search_embedding / np.linalg.norm(search_embedding)
    
    # Prepare documents for BM25
    documents = [node.information for node in information_nodes]
    
    # Tokenize documents and query
    def tokenize(text: str) -> List[str]:
        # Simple tokenization: lowercase and split on non-word characters
        tokens = re.findall(r'\b\w+\b', text.lower())
        return tokens
    
    tokenized_docs = [tokenize(doc) for doc in documents]
    tokenized_query = tokenize(search_term)
    
    # Calculate document frequencies for BM25
    doc_freqs = Counter()
    for doc_tokens in tokenized_docs:
        doc_freqs.update(set(doc_tokens))
    
    num_docs = len(documents)
    avg_doc_length = np.mean([len(tokens) for tokens in tokenized_docs])
    
    # BM25 parameters
    k1 = 1.5
    b = 0.75
    
    def bm25_score(query_tokens: List[str], doc_tokens: List[str]) -> float:
        """Calculate BM25 score for a document given query tokens."""
        score = 0.0
        doc_length = len(doc_tokens)
        doc_token_counts = Counter(doc_tokens)
        
        for term in query_tokens:
            if term in doc_token_counts:
                # Term frequency in document
                tf = doc_token_counts[term]
                # Inverse document frequency
                idf = np.log((num_docs - doc_freqs.get(term, 0) + 0.5) / (doc_freqs.get(term, 0) + 0.5))
                # BM25 formula
                numerator = tf * (k1 + 1)
                denominator = tf + k1 * (1 - b + b * (doc_length / avg_doc_length))
                score += idf * (numerator / denominator)
        
        return score
    
    # Calculate scores for each document
    results = []
    for idx, node in enumerate(information_nodes):
        # Semantic similarity (cosine similarity)
        doc_embedding = np.array(node.embedding)
        doc_embedding_norm = doc_embedding / np.linalg.norm(doc_embedding)
        semantic_score = np.dot(search_embedding_norm, doc_embedding_norm)
        
        # Normalize semantic score to [0, 1] (cosine similarity is already in [-1, 1])
        semantic_score_normalized = (semantic_score + 1) / 2
        
        # Syntactic similarity (BM25) - will normalize after calculating all scores
        syntactic_score = bm25_score(tokenized_query, tokenized_docs[idx])
        
        results.append((node, semantic_score_normalized, syntactic_score))
    
    # Normalize BM25 scores
    if results:
        max_bm25 = max([score for _, _, score in results]) or 1
        min_bm25 = min([score for _, _, score in results]) or 0
        
        # Normalize BM25 scores to [0, 1]
        normalized_results = []
        for node, sem_score, syn_score in results:
            if max_bm25 > min_bm25:
                syntactic_score_normalized = (syn_score - min_bm25) / (max_bm25 - min_bm25)
            else:
                syntactic_score_normalized = 0.0
            
            # Combined score
            combined_score = (semantic_weight * sem_score + 
                        syntactic_weight * syntactic_score_normalized)
            
            if combined_score >= threshold:
                normalized_results.append((node, combined_score, sem_score, syntactic_score_normalized))
        
        results = normalized_results
    else:
        results = []
    
    # Sort by combined score (descending)
    results.sort(key=lambda x: x[1], reverse=True)
    
    # Return only the InformationStructure objects
    return [result[0] for result in results]


In [52]:
# Example usage
results = semantic_search(
    search_term="karcinom prsu",
    information_nodes=information_nodes,
    client=client,
    threshold=0.75,
    semantic_weight=0.75,
    syntactic_weight=0.25
)

# Display results
for result in results:
    print(f"- {result.information}")


- Pacientka má metastatický karcinom prsu.
- Má HER2‑pozitivní karcinom levého prsu.
- Karcinom levého prsu metastazuje do jater.
- Má HER2 pozitivní karcinom levého prsu.
- Má HER2 pozitivní karcinom levého prsu.
- Pacientka má nově diagnostikovaný karcinom levého prsu.
- Karcinom levého prsu metastazuje do skeletu.
- Karcinom levého prsu má diseminaci do jater.
- Karcinom levého prsu se rozšířil do jater.
- Diagnóza je HER2‑pozitivní karcinom levého prsu.
- Pacientka má HER2-pozitivní karcinom levého prsu.
- Pacientka je léčená pro metastatický karcinom prsu.
- Pacientka má HER2 pozitivní karcinom levého prsu.
- Pacientka má HER2 pozitivní karcinom levého prsu.
- Pacientka má HER2‑pozitivní karcinom levého prsu.
- Karcinom levého prsu má metastázy do skeletu.
- Karcinom levého prsu má diseminaci do skeletu.


[InformationStructure(information='Pacientka má nově diagnostikovaný karcinom levého prsu.', embedding=[-0.02277199923992157, -0.00689637241885066, 0.001378004439175129, 0.012122629210352898, -0.006534408312290907, -0.0020177923142910004, 0.013348227366805077, 0.035917017608881, 0.052376870065927505, 0.029515966773033142, 0.031141629442572594, 8.944923320086673e-05, -0.018961848691105843, 0.0182633213698864, 0.010007995180785656, 0.013259324245154858, -0.005038924049586058, -0.02341972477734089, -0.0075440979562699795, -0.05334210768342018, 8.815934415906668e-05, 0.007810808718204498, 0.04417234659194946, -0.013030714355409145, 0.012814805842936039, 0.046509236097335815, 0.0036926709581166506, 0.006851920858025551, -0.006293098442256451, -0.007486945949494839, -0.010420762002468109, 0.02806810848414898, 0.00195905240252614, 0.005518367979675531, 0.02799190580844879, -0.020282700657844543, -0.0005655692075379193, -0.008458534255623817, 0.018860245123505592, 0.009036406874656677, 0.02330